In [16]:
#!/usr/bin/env python
import torch
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [17]:

# ----------------------------
# 1. Load and Preprocess Data
# ----------------------------

# Load the Iris dataset from scikit-learn.
# The dataset has 150 samples, each with 4 features.
# There are 3 classes (0, 1, 2) that we want to classify.
iris = datasets.load_iris()
X = iris.data      # shape: (150, 4)
y = iris.target    # shape: (150,)

# Set seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Standardize features (zero mean, unit variance) to improve training performance.
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Convert NumPy arrays to PyTorch tensors.
# We use torch.float32 for input features and torch.long for labels.
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)

# Optionally, split the data into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [18]:
# ----------------------------
# 2. Initialize Network Parameters
# ----------------------------

# Define the network architecture.
input_size = 4     # Four features from the iris dataset.
hidden_size = 8   # Number of neurons in the hidden layer (an arbitrary choice).
output_size = 3    # Three output classes.

# Manually initialize weights and biases.
# We use basic tensors with requires_grad=True so that PyTorch tracks operations for automatic differentiation.
W1 = torch.randn(input_size, hidden_size, dtype=torch.float32, requires_grad=True)
b1 = torch.randn(hidden_size, dtype=torch.float32, requires_grad=True)
W2 = torch.randn(hidden_size, output_size, dtype=torch.float32, requires_grad=True)
b2 = torch.randn(output_size, dtype=torch.float32, requires_grad=True)

In [ ]:
W1

In [20]:
# ----------------------------
# 3. Define the Forward Pass
# ----------------------------
def forward(x):
    """
    Perform a forward pass through the network.
    
    A forward pass means taking the input data and computing the output of the network.
    Here, we first compute a linear transformation using W1 and b1, apply a non-linear ReLU activation,
    then compute a second linear transformation using W2 and b2 to get the final raw output scores (logits).
    """
    # First layer: linear transformation (matrix multiplication) followed by bias addition.
    z1 = torch.mm(x, W1) + b1  # Shape: (n_samples, hidden_size)
    
    # ReLU activation: sets negative values to zero.
    a1 = z1.clamp(min=0)
    
    # Second layer: another linear transformation producing logits.
    logits = torch.mm(a1, W2) + b2  # Shape: (n_samples, output_size)
    return logits

def softmax(logits):
    """
    Compute the softmax of the logits to obtain class probabilities.
    
    Softmax converts raw scores into probabilities that sum to 1.
    We subtract the max for numerical stability.
    """
    exp_logits = torch.exp(logits - torch.max(logits, dim=1, keepdim=True)[0])
    probs = exp_logits / torch.sum(exp_logits, dim=1, keepdim=True)
    return probs

def cross_entropy_loss(logits, y_true):
    """
    Compute the cross-entropy loss between the predicted logits and the true labels.
    
    The cross-entropy loss measures the difference between the predicted probability distribution (obtained by softmax)
    and the true distribution (which assigns a probability of 1 to the correct class). Lower loss indicates better predictions.
    """
    probs = softmax(logits)
    n_samples = logits.shape[0]
    # For each sample, select the probability corresponding to the true label and take the negative log.
    correct_logprobs = -torch.log(probs[range(n_samples), y_true])
    loss = torch.sum(correct_logprobs) / n_samples
    return loss

In [ ]:
# ----------------------------
# 4. Training the Network
# ----------------------------

# Hyperparameters for training.
learning_rate = 0.01
num_epochs = 500

# Training loop.
for epoch in range(num_epochs):
    # ----- Forward Pass -----
    # Compute the raw predictions (logits) for the current batch.
    logits = forward(X_train)
    
    # Compute the loss between the predictions and the true labels.
    loss = cross_entropy_loss(logits, y_train)
    
    # ----- Backward Pass -----
    # A backward pass computes the gradients of the loss with respect to each parameter.
    # It applies the chain rule to propagate the error backwards through the network.
    loss.backward()
    
    # ----- Update Parameters -----
    # We update each parameter using gradient descent: new_param = old_param - learning_rate * gradient
    with torch.no_grad():
        W1 -= learning_rate * W1.grad
        b1 -= learning_rate * b1.grad
        W2 -= learning_rate * W2.grad
        b2 -= learning_rate * b2.grad
    
    # Zero the gradients after updating so that they do not accumulate.
    W1.grad.zero_()
    b1.grad.zero_()
    W2.grad.zero_()
    b2.grad.zero_()
    
    # Print the loss every 50 epochs for monitoring.
    if (epoch + 1) % 50 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

In [ ]:
# ----------------------------
# 5. Evaluate the Model
# ----------------------------
with torch.no_grad():
    # Compute predictions on the test set.
    test_logits = forward(X_test)
    test_probs = softmax(test_logits)
    # Choose the class with the highest probability.
    predictions = torch.argmax(test_probs, dim=1)
    
    # Calculate accuracy.
    accuracy = (predictions == y_test).float().mean()
    print(f"\nTest Accuracy: {accuracy.item() * 100:.2f}%")